In [ ]:
# analyse a pdf file
# read the file then transform to pandas dataFrame
# split the text into sentenses
# cleaning the data , and set them in consistence Format
# add new colomuns , for the length of characters and words in each sentences
# set the  Average Word Length
# use CountVectorizer to build a matrix of token counts
# use TfidfVectorizer to calculate the Frequency of each token in the Text
# use n gramm to represent sequences of words or characters in a text document. 
# get the commun used word in the text
# set a trigram features , to know the most commun words

In [1]:
import PyPDF2 #  need it to read the pdf file
import pandas as pd # need it to tranforom into cdv file 

In [ ]:
import PyPDF2
import spacy

# Open the PDF file
pdf_file = open('Chan2014a.pdf', 'rb')

# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Extract text from the PDF file
text = ''

# for page_num in range(pdf_reader.getNumPages()):
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()
    
    
    # split the text into sentences
    sentences = text.split('. ')

# create a pandas DataFrame from the sentences
df = pd.DataFrame(sentences, columns=['Sentences'])
# df_t = pd.DataFrame(sentences, columns=['text'])
pdf_file.close()
# write the DataFrame to a CSV file
df.to_csv('output_file.csv', index=False)
display(df)

In [4]:
# Remove none text characters from text column
df["Sentences"] = df["Sentences"].str.replace("[^a-zA-Z]", " ")
df["Sentences"][:5]

C:\Users\NIZAR\AppData\Local\Temp/ipykernel_40392/1531420399.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Sentences"] = df["Sentences"].str.replace("[^a-zA-Z]", " ")


0       Thirty Fifth International Conference on In...
1    The exponential gro wth of this nascent online...
2    Despite this importance   limited effort has b...
3    Using a novel proprietary  dataset from a lead...
4    After accounting fo r endogeneity via a matche...
Name: Sentences, dtype: object

In [6]:
# Made all charactuers lowercase
df["Sentences"] = df["Sentences"].str.lower()
df["Sentences"][0]

'   thirty fifth international conference on informati on systems  auckland        hiring biases in online labor markets     the case of gender stereotyping   completed research paper      jason chan   carlson school of management   university of minnesota         th  ave south  minneapolis    mn        united states  jchancf umn edu jing wang   school of business and management   hkust  lee shau kee business building    clear water bay  kowloon  hong kong  jwang ust hk    abstract  online labor marketplaces facilitate the efficient matching of employers and workers  across geographical boundaries'

In [7]:
# Length of Characters
df["Char Len"] = df["Sentences"].str.len()
df[:5]

,Sentences,Char Len
0,thirty fifth international conference on in...,603
1,the exponential gro wth of this nascent online...,108
2,despite this importance limited effort has b...,174
3,using a novel proprietary dataset from a lead...,147
4,after accounting fo r endogeneity via a matche...,188


In [8]:
# Length of Words
df["Words Len"] = df["Sentences"].str.split().str.len()
df[:5]

,Sentences,Char Len,Words Len
0,thirty fifth international conference on in...,603,79
1,the exponential gro wth of this nascent online...,108,16
2,despite this importance limited effort has b...,174,27
3,using a novel proprietary dataset from a lead...,147,23
4,after accounting fo r endogeneity via a matche...,188,29


In [9]:
# Average Word Length
df["Avg Word Len"] = (df["Char Len"] / df["Words Len"]).round(2)
df[:2]

,Sentences,Char Len,Words Len,Avg Word Len
0,thirty fifth international conference on in...,603,79,7.63
1,the exponential gro wth of this nascent online...,108,16,6.75


In [10]:
#Initialize Vectorizer
# CountVectorizer is a text feature extraction tool provided by scikit-learn that can be used 
# to convert a collection of text documents into a matrix of token counts. 
# This matrix can then be used for various text analysis tasks such as text classification, clustering, or information retrieval.
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df = 0.1, max_df = 0.9)

In [12]:
cv_transform = cv.fit_transform(df["Sentences"])
cv_transform[:700]

<638x16 sparse matrix of type '<class 'numpy.int64'>'
	with 2029 stored elements in Compressed Sparse Row format>

In [13]:
# A unique token is a token type that occurs only once in a given corpus of text, 
# whereas a token is an instance of a token type that occurs one or more times in the corpus.

cv_df = pd.DataFrame(cv_transform.toarray(),
                     columns = cv.get_feature_names()).add_prefix("Count_")
cv_df[:5]

,Count_and,Count_are,Count_for,Count_gender,Count_hiring,Count_in,Count_is,Count_labor,Count_of,Count_on,Count_online,Count_that,Count_the,Count_to,Count_we,Count_workers
0,2,0,0,1,1,1,0,2,5,2,2,0,2,0,0,1
1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0
2,1,0,0,0,2,1,0,1,0,0,1,0,0,1,0,0
3,0,0,0,1,1,1,0,1,1,1,1,0,1,0,1,0
4,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1


In [14]:
speech_df = pd.concat([df, cv_df], axis = 1)
speech_df.shape

(638, 20)

In [15]:
# TF-IDF is calculated by multiplying the term frequency (TF) of a term in a document 
# by its inverse document frequency (IDF) across all documents in the corpus. 
# The term frequency measures how often a term appears in a document,
# while the inverse document frequency measures how rare or unique a term is across all documents in the corpus.
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 100, stop_words = "english")
print(tv)

TfidfVectorizer(max_features=100, stop_words='english')


In [16]:
tv_transformed = tv.fit_transform(df["Sentences"])

tv_df = pd.DataFrame(tv_transformed.toarray(),
                     columns = tv.get_feature_names()).add_prefix("TFIDF_")

tvdf_trans = pd.concat([df, tv_df], axis = 1)
tvdf_trans[:5]

,Sentences,Char Len,Words Len,Avg Word Len,TFIDF_al,TFIDF_applicant,TFIDF_applicants,TFIDF_attributes,TFIDF_auckland,TFIDF_based,...,TFIDF_use,TFIDF_used,TFIDF_using,TFIDF_utility,TFIDF_value,TFIDF_variable,TFIDF_women,TFIDF_worker,TFIDF_workers,TFIDF_working
0,thirty fifth international conference on in...,603,79,7.63,0.0,0.0,0.0,0.0,0.201833,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.142481,0.0
1,the exponential gro wth of this nascent online...,108,16,6.75,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,despite this importance limited effort has b...,174,27,6.44,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,using a novel proprietary dataset from a lead...,147,23,6.39,0.0,0.0,0.0,0.0,0.000000,0.362069,...,0.0,0.0,0.356244,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,after accounting fo r endogeneity via a matche...,188,29,6.48,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.247409,0.0


In [17]:
# Isolate the row to be examined
sample_row = tv_df.iloc[0]
# Print the top 5 words of the sorted output
print(sample_row.sort_values(ascending=False).head())


TFIDF_management      0.455590
TFIDF_labor           0.286252
TFIDF_online          0.247873
TFIDF_stereotyping    0.223585
TFIDF_research        0.223585
Name: 0, dtype: float64


In [18]:
# The TF-IDF weighting scheme is used to measure the importance or relevance of a term 
# (i.e., a word or phrase) in a document, 
# relative to its frequency in the entire corpus of documents.
tv = TfidfVectorizer(max_features = 100, stop_words = "english")

# Fit the vectroizer and transform the data
tv_transformed = tv.fit_transform(df["Sentences"])

# Transform  data
tv_transformed = pd.DataFrame(tv_transformed.toarray(),
                                   columns = tv.get_feature_names()).add_prefix("Train_TFIDF_")
tv_transformed.head()


,Train_TFIDF_al,Train_TFIDF_applicant,Train_TFIDF_applicants,Train_TFIDF_attributes,Train_TFIDF_auckland,Train_TFIDF_based,Train_TFIDF_bias,Train_TFIDF_biases,Train_TFIDF_caliper,Train_TFIDF_category,...,Train_TFIDF_use,Train_TFIDF_used,Train_TFIDF_using,Train_TFIDF_utility,Train_TFIDF_value,Train_TFIDF_variable,Train_TFIDF_women,Train_TFIDF_worker,Train_TFIDF_workers,Train_TFIDF_working
0,0.0,0.0,0.0,0.0,0.201833,0.000000,0.000000,0.177822,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.142481,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.496865,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.362069,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.356244,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.299468,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.247409,0.0


In [19]:
# n gramm
# represent sequences of words or characters in a text document. 
# An n-gram is a contiguous sequence of n items from a given sequence of text, 
# where an item can be a word, a character, or a subword unit.
# for Train_set
tv_bi_gram_vec = TfidfVectorizer(ngram_range = (4, 8))

tv_bi_gram = tv_bi_gram_vec.fit_transform(df["Sentences"])

tv_bi_gram_vec.get_feature_names()[300:310]

['act of workers gender on hiring',
 'act of workers gender on hiring outcomes',
 'act of workers gender on hiring outcomes in',
 'actions implicate the income',
 'actions implicate the income of',
 'actions implicate the income of millions',
 'actions implicate the income of millions of',
 'actions implicate the income of millions of workers',
 'active projects and workers',
 'actual working hours online']

In [60]:
# Create DataFrame with Count Features
tv_df = pd.DataFrame(tv_bi_gram.toarray(),
                     columns = tv_bi_gram_vec.get_feature_names()).add_prefix("Counts_")

tv_sums = tv_df.sum()

tv_sums.head()

Counts_ab le to infer                              0.077951
Counts_ab le to infer the                          0.077951
Counts_ab le to infer the gender                   0.077951
Counts_ab le to infer the gender information       0.077951
Counts_ab le to infer the gender information of    0.077951
dtype: float64

In [63]:
# Common Words
tv_sums.sort_values(ascending = False)[:5]

Counts_quarterly journal of economics              4.118816
Counts_journal of applied psychology               2.000000
Counts_journal of economic perspectives            2.000000
Counts_the american economic review                2.000000
Counts_thirty fifth international conference on    1.000677
dtype: float64

In [65]:
#Using longer n-grams
# Instantiate a trigram vectorizer
cv_trigram_vec = CountVectorizer(max_features = 100,
                                 stop_words = "english",
                                 ngram_range = (3, 5))

# Fit and apply trigram vectorizer
cv_trigram = cv_trigram_vec.fit_transform(df["Sentences"])

# Print the trigram features
print(cv_trigram_vec.get_feature_names())

['american economic review', 'biases online labor', 'biases online labor markets', 'biases online labor markets thirty', 'checkbld checkbld checkbld', 'checkbld checkbld checkbld checkbld', 'checkbld checkbld checkbld checkbld checkbld', 'checkbld checkbld log likelihood', 'checkbld checkbld log likelihood observations', 'checkbld checkbld matched', 'checkbld checkbld matched workers', 'checkbld checkbld matched workers job', 'checkbld log likelihood', 'checkbld log likelihood observations', 'checkbld matched workers', 'checkbld matched workers job', 'checkbld matched workers job posting', 'conditional logistic regressions', 'conference informatio systems', 'conference informatio systems auckland', 'economics value thirty', 'economics value thirty fifth', 'economics value thirty fifth international', 'feminine neutral masculine', 'fifth international conference', 'fifth international conference informatio', 'fifth international conference informatio systems', 'hiring biases online', 'h

In [66]:
# finding the most commun word
cv_tri_df = pd.DataFrame(cv_trigram.toarray(),
                         columns = cv_trigram_vec.get_feature_names()).add_prefix("Counts_")

# Print the top 5 words in the sorted output
cv_tri_df.sum().sort_values(ascending = False).head()

Counts_online labor markets                           25
Counts_thirty fifth international                     17
Counts_thirty fifth international conference          17
Counts_fifth international conference                 17
Counts_international conference informatio systems    16
dtype: int64